## Lab 3

<h4>
1. Run a simple bivariate regression, and interpret your results. (Did the results fit your expectations? Why? Why not?)

2. Add an additional variable that might mediate or partly "explain" the initial association from that simple regression above -- and explain your results. Did it work out? Yes? No?

3. Run another multiple regression. Tell me how you expect your dependent variable to be affected by the independent variables. Interpret your results.

4. Now add another independent variable to that model in Question 3, specifically a set of dummy variables. Tell me why you added that new set of variables and what effect you expected them to have. Did they have an effect? Interpret that new model. 
    
</h4>

In [2]:
from __future__ import division
import pandas as pd
import numpy as np
import re, os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns #os.system('pip install seaborn')
from scipy import stats
import pycountry_convert as pc #os.system("pip install pycountry-convert")
from colour import Color #os.system('pip install colour')

def printurn(x):
    print(x)
    return(x)

def recode_onehot_if(s='CHILDRENS COAT', pattern='CHILDREN'):
    result = '1' if pattern in str(s) else '0'
    return(result)

def check_color(color):
    try:
        Color(color)
        return(True)
    except ValueError as e:
        return(False)

def extract_color(pdcolumn):
    pdcolumn = pd.Series(pdcolumn.copy()) if type(pdcolumn)==str else pd.Series(pdcolumn) if type(pdcolumn)==list else pdcolumn
    newcolumn = pdcolumn.copy().apply(lambda s: ' '.join([i for i in str(s).split(' ') if check_color(i)]))
    return(newcolumn)

def compute_percentage(x, my_crosstab):
      pct = float(x/my_crosstab['count'].sum()) * 100
      return round(pct, 2)

os.listdir('../lab1')

['.ipynb_checkpoints',
 'courseworks',
 'data.csv',
 'lab 1 updated.html',
 'lab1.ipynb',
 'lab1.py']

In [9]:

try: # NOTE: trying to read the csv file with basic default args first, but if error, we're adding the encoding argument. Since this dataset has text data, the encoding has some quirks and some Python & Pandas versions don't support it with default UTF-8 encoding.
    df = pd.read_csv('../lab1/data.csv').drop_duplicates().query('Quantity>=0 and Quantity<=70000 and UnitPrice>=0').reset_index(drop=True) # dataset is from #https://www.kaggle.com/carrie1/ecommerce-data
except:
    df = pd.read_csv('../lab1/data.csv', encoding="ISO-8859-1").drop_duplicates().query('Quantity>=0 and Quantity<=1000 and UnitPrice>=0').reset_index(drop=True) #https://www.kaggle.com/carrie1/ecommerce-data 
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
525932,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
525933,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
525934,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
525935,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [10]:
df_new = df.copy().apply(lambda y: y.str.lower() if str(y.dtype)=='object' else y).assign(
    purchase_price = lambda d: d.UnitPrice * d.Quantity,
    invoice_month = lambda d: pd.DatetimeIndex(d.InvoiceDate).month,
    invoice_year = lambda d: pd.DatetimeIndex(d.InvoiceDate).year,
    invoice_day_of_month = lambda d: pd.DatetimeIndex(d.InvoiceDate).day,
    n_words_description = lambda d: [len(str(x).split()) for x in d.Description],
    material=lambda d: d.Description.str.extract(r'(ceramic|metal|wood|plastic|cloth|felt|fabric|silk|polyester|linen|cotton|compostable|mahogany|canvas|cashmere|chiffon|denim|viscose|wool|fur|lace|leather|diamond|crystal|rhinestone|jewel|birch|rubber|wax|vintage|organic|pashmina|satin|spandex|suede|cement|marble)', expand=False, flags=re.IGNORECASE),
    color=lambda d: extract_color(d.Description).apply(lambda s: s.split(' ')[0]),
    ).apply(lambda y: y.str.lower().str.strip().replace('', np.nan) if str(y.dtype)=='object' else y)
print('df_new.shape # (531283, 11)', df_new.shape) # (531283, 11))
new_list = []
for boob in df_new.Country.str.title():
    try:
        new_boob = pc.country_name_to_country_alpha2(boob, cn_name_format="default")
        new_boob = pc.country_alpha2_to_continent_code(new_boob)
    except:
        new_boob = ''
    new_list.append(new_boob)
df_new['continent'] = new_list
df_new




df_new.shape # (531283, 11) (525937, 15)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,purchase_price,invoice_month,invoice_year,invoice_day_of_month,n_words_description,material,color,continent
0,536365,85123a,white hanging heart t-light holder,6,12/1/2010 8:26,2.55,17850.0,united kingdom,15.30,12,2010,1,5,NaN,white,EU
1,536365,71053,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,united kingdom,20.34,12,2010,1,3,metal,white,EU
2,536365,84406b,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,united kingdom,22.00,12,2010,1,5,NaN,NaN,EU
3,536365,84029g,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,united kingdom,20.34,12,2010,1,6,NaN,NaN,EU
4,536365,84029e,red woolly hottie white heart.,6,12/1/2010 8:26,3.39,17850.0,united kingdom,20.34,12,2010,1,5,wool,red,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525932,581587,22613,pack of 20 spaceboy napkins,12,12/9/2011 12:50,0.85,12680.0,france,10.20,12,2011,9,5,NaN,NaN,EU
525933,581587,22899,children's apron dolly girl,6,12/9/2011 12:50,2.10,12680.0,france,12.60,12,2011,9,4,NaN,NaN,EU
525934,581587,23254,childrens cutlery dolly girl,4,12/9/2011 12:50,4.15,12680.0,france,16.60,12,2011,9,4,NaN,NaN,EU
525935,581587,23255,childrens cutlery circus parade,4,12/9/2011 12:50,4.15,12680.0,france,16.60,12,2011,9,4,NaN,NaN,EU


In [20]:
print(df_new.material.value_counts().head())
# Checking value counts (frequencies) of words in the description field
pd.Series(' '.join(df_new.Description.astype(str).unique().tolist()).split()).value_counts()#.head(20)


vintage    32992
wood       25234
metal      20936
felt        9586
ceramic     8769
Name: material, dtype: int64


set           322
pink          298
of            240
heart         236
vintage       219
             ... 
frieze          1
croquet         1
czech           1
poinsettia      1
lazer           1
Length: 2412, dtype: int64

In [58]:
# df_new.query('material=="vintage"')
df_new['bulk_item_indicator'] = np.where(df_new.Description.astype(str).str.contains('\\b(set|pack|box of|crate of)\\b'), 'Yes', 'No')

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


#### 1. Run a simple bivariate regression, and interpret your results. (Did the results fit your expectations? Why? Why not?)


In [96]:
br_fashion1 = smf.ols(formula = 'UnitPrice~bulk_item_indicator', data = df_new).fit()
print(br_fashion1.summary())

                            OLS Regression Results                            
Dep. Variable:              UnitPrice   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     60.39
Date:                Sun, 30 May 2021   Prob (F-statistic):           7.79e-15
Time:                        14:55:12   Log-Likelihood:            -2.6318e+06
No. Observations:              525937   AIC:                         5.264e+06
Df Residuals:                  525935   BIC:                         5.264e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

### Interpretation: the bivariate regression does  fit our expectations because the p-value is 0.000, showcasing that the regression is statistically significant. There is a negative relationship between UnitPrice and bulk_item_indicator, such that when bulk_item_indicator is yes, the price is lower by $1.0747. (p<0.001) 

### 2. Add an additional variable that might mediate or partly "explain" the initial association from that simple regression above -- and explain your results. Did it work out? Yes? No?

In [90]:
br_fashion2 = smf.ols(formula = 'UnitPrice~bulk_item_indicator+Quantity', data = df_new).fit()
print(br_fashion2.summary())

                            OLS Regression Results                            
Dep. Variable:              UnitPrice   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     126.8
Date:                Sun, 30 May 2021   Prob (F-statistic):           8.47e-56
Time:                        14:49:20   Log-Likelihood:            -2.6317e+06
No. Observations:              525937   AIC:                         5.263e+06
Df Residuals:                  525934   BIC:                         5.263e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

### Items that already come in multi-packs are probably only purchased in lower quantities because one bulk pack of many items is probably enough. Quantity of units purchased is the mediator for the relationship between whether multipack items are higher or lower price. From the results above, it contradicted our initial hypothesis as items that came in multi-packs, as established by bulk_item_indicator, was purchased in higher quantities despite the unit price of multipack items being higher. 

## 3. Run another multiple regression. Tell me how you expect your dependent variable to be affected by the independent variables. Interpret your results.


#### Perhaps invoice month predicts quantity of items purchased, but controlling for whether the item comes in a multipack (bulk_item_indicator) because an item being a multipack might confound the relationship between invoice month and Quantity. I'm thinking invoice month might be related to quantity because people might make more purchases later in the year where all the big gifting holidays happen.


In [46]:
br_fashion3 = smf.ols(formula = 'Quantity~invoice_month+bulk_item_indicator', data = df_new).fit()
print(br_fashion3.summary())

## 4. Now add another independent variable to that model in Question 3, specifically a set of dummy variables. Tell me why you added that new set of variables and what effect you expected them to have. Did they have an effect? Interpret that new model. 

In [84]:
# df_new['continent_eu'] = np.where(df_new.continent=='EU', 1, 0)
# df_new['continent_as'] = np.where(df_new.continent=='AS', 1, 0)
# df_new['continent_oc'] = np.where(df_new.continent=='OC', 1, 0)
# df_new['continent_sa'] = np.where(df_new.continent=='SA', 1, 0)
# df_new['continent_na'] = np.where(df_new.continent=='NA', 1, 0)
# df_new['continent_unknown'] = np.where(df_new.continent=='', 1, 0)
# df_new['country_usa'].value_counts()
# print(df_new.Country.value_counts())
print(df_new.continent.value_counts())
print(df_new['continent_eu'].value_counts())
df_new.iloc[:,15:]


continents_list = ['EU', 'AS', 'OC', 'SA', 'NA', '']

for each_continent_code in continents_list:
    df_new['continent_' + re.sub('^$', 'unknown', each_continent_code)] = np.where(df_new.continent==each_continent_code, 1, 0)

    
df_new.iloc[:,20:]

EU    513347
        9368
AS      1856
OC      1183
NA       151
SA        32
Name: continent, dtype: int64
1    513347
0     12590
Name: continent_eu, dtype: int64


,continent_as,continent_oc,continent_na,continent_sa,continent_unknown,continent_EU,continent_AS,continent_OC,continent_SA,continent_NA,continent_
0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
525932,0,0,0,0,0,1,0,0,0,0,0
525933,0,0,0,0,0,1,0,0,0,0,0
525934,0,0,0,0,0,1,0,0,0,0,0
525935,0,0,0,0,0,1,0,0,0,0,0


In [87]:
br_fashion4 = smf.ols(formula = 'Quantity~invoice_month+bulk_item_indicator+continent_AS+continent_EU+continent_OC+continent_NA+continent_SA', data = df_new).fit()
print(br_fashion4.summary())

                            OLS Regression Results                            
Dep. Variable:               Quantity   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     917.0
Date:                Sun, 30 May 2021   Prob (F-statistic):               0.00
Time:                        02:01:28   Log-Likelihood:            -2.5085e+06
No. Observations:              525937   AIC:                         5.017e+06
Df Residuals:                  525929   BIC:                         5.017e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           